In [1]:
from nltk.tokenize import sent_tokenize
import pandas as pd
import glob
import re

In [2]:
bio=pd.read_csv("bio_auth_books.csv")
bio2=pd.read_csv("bio_auth_books2.csv")
bio=pd.concat((bio,bio2),axis=0)

In [7]:
bio=bio.drop_duplicates(subset=["title"])

In [39]:
f = open("desc.txt", "r", encoding="utf-8")
text=f.read()
f.close()
text=text.replace('\r', ' ').replace('\n', ' ')
text=text.split("PREFACE TO THE SECOND EDITION.")[2]
text=text.split("probability to the action of sexual selection.")[0]
text+="probability to the action of sexual selection."
text=text.strip()
dom = filt(sent_tokenize(text))

In [37]:
f = open("oos.txt", "r", encoding="utf-8")
text=f.read()
f.close()
text=text.replace("\n"," ")
text=text.split("Sixth London Edition, with all Additions and Corrections.")[1]
text=text.split("from so simple a beginning endless forms most beautiful and most wonderful have been, and are being evolved.")[0]
text=text.strip()
oos = filt(sent_tokenize(text))

In [8]:
def filt(sents):
    ret=[]
    for s in sents:
        if len(s)>15:
            ret.append(s)
    return ret
def clean(str):

    text = str.replace('\r', ' ').replace('\n', ' ')
    #text=re.sub(r'[^a-zA-Z. ]+', '', text)
    text=" ".join(text.split())
    text=text.strip()
    return text

In [34]:

for id in bio["id"]:
    path=glob.glob('bio/'+id+'/*djvu.txt')
    try:
        f = open(path[0], "r", encoding="utf-8")
        text=f.read()
    except:
        continue
    
    if len(text)<1000:
        continue
    text=clean(text)
    out = open("clean_bio/"+id+".txt", "w", encoding="utf-8")
    out.write(text)
    f.close() 


In [2]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('thenlper/gte-small')
import pickle
from scipy.spatial.distance import cdist
import numpy as np

In [4]:
from sentence_transformers.util import cos_sim


In [5]:
sentences = ['I at least have so much to do in unraveling certain human lots, and seeing how they were woven and interwoven, that all the light I can command must be concentrated on this particular web, and not dispersed over that tempting range of relevancies called the universe.', 'We shall never, probably, disentangle the inextricable web of affinities between the members of any one class; but when we have a distinct object in view, and do not look to some unknown plan of creation, we may hope to make sure but slow progress.']

embeddings = model.encode(sentences)
print(cos_sim(embeddings[0], embeddings[1]))

tensor([[0.8547]])


In [ ]:
file = open('vec_oos.pickle', 'rb')
emb_oos = pickle.load(file)
file.close()
file = open('vec_dom.pickle', 'rb')
emb_dom = pickle.load(file)
file.close()
emb_oos=np.array(emb_oos)
emb_dom=np.array(emb_dom)


In [11]:
import os

In [ ]:
for id in bio["id"]:
    if os.path.exists('bio_vectors/'+id+'.pickle'):
        continue
    path=glob.glob('clean_bio/'+id+'.txt')
    try:
        f = open(path[0], "r", encoding="utf-8")
        text=f.read()
    except:
        continue
        

    f.close()
    sents = filt(sent_tokenize(text))
    emb = model.encode(sents) 
    emb=np.array(emb)
    
    with open('bio_vectors/'+id+'.pickle', 'wb') as handle:
        pickle.dump(emb, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
        

    

In [ ]:

for comp in ["oos", "dom"]:
    if comp=="oos":
        other=emb_oos
    else:
        other=emb_dom 

    match=cdist(emb, other, "cosine")
    match=np.asarray((match<0.1)).nonzero()

In [ ]:
letter